# Imports

In [ ]:
!pip install turicreate

In [ ]:
import os
import numpy as np
import pandas as pd
import turicreate
from zipfile import ZipFile

# Load Data

In [ ]:
#zip dir
ds_dir = '../input/instacart-market-basket-analysis'

In [ ]:
#unzip dataset
with ZipFile(os.path.join(ds_dir,"aisles.csv.zip"), 'r') as zipObj:
   zipObj.extractall()
with ZipFile(os.path.join(ds_dir,"departments.csv.zip"), 'r') as zipObj:
   zipObj.extractall()
with ZipFile(os.path.join(ds_dir,"order_products__prior.csv.zip"), 'r') as zipObj:
   zipObj.extractall()
with ZipFile(os.path.join(ds_dir,"order_products__train.csv.zip"), 'r') as zipObj:
   zipObj.extractall()
with ZipFile(os.path.join(ds_dir,"orders.csv.zip"), 'r') as zipObj:
   zipObj.extractall()
with ZipFile(os.path.join(ds_dir,"products.csv.zip"), 'r') as zipObj:
   zipObj.extractall()
with ZipFile(os.path.join(ds_dir,"sample_submission.csv.zip"), 'r') as zipObj:
   zipObj.extractall()

In [ ]:
#load data
prior = pd.read_csv('order_products__prior.csv')
train = pd.read_csv('order_products__train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

#fillna
prior.fillna(0,inplace=True)
train.fillna(0,inplace=True)
orders.fillna(0,inplace=True)

In [ ]:
#ignore prior and train, merge them all
data = pd.concat([train,prior])
ou = orders[orders.eval_set != 'test']
data = pd.merge(data,ou,on='order_id').drop('order_id',axis=1)
data.drop('eval_set',axis=1,inplace=True)

In [ ]:
#convert reordered to rating/score
reordered = data.groupby(['user_id','product_id']).reordered.sum()
data = pd.merge(data,reordered, on=['user_id','product_id'], how='left')
data = data[data.reordered_y>0]
data['target'] = np.log(data['reordered_y']+1)
data.drop(['reordered_x','reordered_y','add_to_cart_order'],axis=1,inplace=True)

In [ ]:
#set test dataset
test = orders[orders.eval_set=='test'].drop(['eval_set','order_id'],axis=1)

In [ ]:
#set relation dataset to exclude non recurring items
relation = data[['user_id','product_id']].copy()
relation.drop_duplicates(inplace=True)

In [ ]:
#backup to trace nan data
sub = orders[orders.eval_set == 'test']
sub = sub[['order_id','user_id']]

In [ ]:
data.head(1)

In [ ]:
test.head(1)

# Convert to turicreate dataframe

In [ ]:
train_data = turicreate.SFrame(data)
test_data = turicreate.SFrame(test)
content_data = turicreate.SFrame(products.drop('product_name',axis=1))
relation_data = turicreate.SFrame(relation)

# Training

In [ ]:
#training
recommender =  turicreate.recommender.factorization_recommender.create(train_data, user_id='user_id', item_id='product_id', target='target', solver='sgd', user_data=None, item_data=content_data, verbose=True)

# Predicting

In [ ]:
def clean_prediction(row):
    data = row.products
    data = str("".join(str(data))[1:-1].replace(',',' '))
    return data

In [ ]:
#predicting
result = recommender.recommend(users=test_data, items=relation_data, exclude_known=False, k=len(products)).to_dataframe()

In [ ]:
thresholds = {
    'mean': result.score.mean(),
    'std' : result.score.std(),
    'zero' : 0,
    'q05' : result.score.quantile(q=0.05),
    'q10' : result.score.quantile(q=0.1)
    }

results = {}

In [ ]:
for key,item in thresholds.items():
    print(key,':',str(item))
    results[key] = result[result['score'] > item]
    results[key] = results[key].groupby('user_id')['product_id'].apply(list).reset_index(name='products')
    results[key]['products'] = results[key].apply(clean_prediction, axis=1)

# Fill nan with None

In [ ]:
for key,item in thresholds.items():
    results[key] = pd.merge(sub, results[key],how='outer',on='user_id').sort_values('user_id')
    results[key].fillna('None', inplace=True)
    results[key].drop('user_id',axis=1,inplace=True)

In [ ]:
for key,item in thresholds.items():
    results[key].to_csv('submission_turi_fr_adagrad_'+str(key)+'.csv',index=False)